In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)

In [2]:
#### CONSTANTS ####
#pixel to distance ratio, 1 pixel = 16 µm, x100 lens
pixel_to_distance=16/100
#duration simulation in seconds
tot_time=200
#length of each time step in seconds
dt=0.001
#number of steps to run the simulation for
nsteps = int(tot_time/dt)
#Thermal diffusivity, D0/Dbulk = 0.48 (Rg/h)^(-2/3)
#https://doi.org/10.1073/pnas.0605900103 
D_bulk=0.46
Rg=0.69
hbar=0.2
D0=0.48*(Rg/hbar)** (-2/3) * D_bulk
#Drag coefficient, drag=KbT/D0
drag=1/D0
#obtain a and b for potential from known energy barrier 6.1 kbT. 
#change b to change the shape.
a=4*6.1
b=a 

In [3]:
###Functions###
def pressure(amplitude, T, t):
    # periodic pressure
    return amplitude*np.cos(2*np.pi/T*t)

def Langevin(x, t, amplitude, T, sigma,random_number):
    # Langevin equation
    
    #potential term
    negative_derivative_potential = a*x - b*(x**3)
    
    #pressure term
    #exp camera: 1 pixel = 16 µm, x100 lens
    #linear relationship bewtween pressure and velocity, y=0.2028x-0.178, omitted b term -0.178
    #referenced Shayan Master thesis p.34
    F_p = pixel_to_distance*(0.2028*pressure(amplitude, T, t) )
    
    #D linear to variance of electro noise, includes thermal diffusion
    #referenced Stein paper p.3
    D_eff = D0 + (1.82*10**-3 * sigma**2)
    effective_noise = np.sqrt(2*D_eff/dt)*random_number
    
    return negative_derivative_potential/drag + F_p + effective_noise

def SR(amplitude, T, sigma):
    #solving Langevin
    
    # create empty lists to store values
    x = np.zeros(nsteps, np.longdouble)
    t = np.zeros(nsteps, np.longdouble)

    # set initial conditions at time zero
    x[0] = -np.sqrt(a/b)
    t[0] = 0
    
    # RK4
    for ii in range(nsteps-1):
        #keep the random number same for all k's
        random_number=np.random.normal(0,1)
        k1 = dt*Langevin(x[ii],t[ii],amplitude,T,sigma,random_number)
        k2 = dt*Langevin(x[ii]+0.5*k1,t[ii]+dt/2,amplitude,T,sigma,random_number)
        k3 = dt*Langevin(x[ii]+0.5*k2,t[ii]+dt/2,amplitude,T,sigma,random_number)
        k4 = dt*Langevin(x[ii]+0.5*k3,t[ii]+dt/2,amplitude,T,sigma,random_number)

        # update value of Q at each time step and step time by dt
        x[ii+1]=x[ii]+(k1+2*k2+2*k3+k4)/6
        t[ii+1]=t[ii]+dt
    
    #change to np.array
    x=np.array(x)
    t=np.array(t)
    
    return (t,x)

In [4]:
#counting number of jumps
def counting(x):
    crossings = np.zeros(len(x))
    #set upper and lower bounds
    upper = (0.5)
    lower = (-0.5)
    i = 0
    while i< len(x)-1:
        #case 1: lower to upper
        if (x[i] < lower and x[i+1]> upper):
            crossings[i] = 1
            i += 1

        #case 2: upper to lower
        elif (x[i] > upper and x[i+1] < lower):
            crossings[i] = 1
            i += 1

        #case 3: upper to middle to lower
        elif (x[i] > upper and x[i+1]< upper):
            j = i+1
            while j < len(x)-1:
                #if middle to lower
                if (x[j] > lower and x[j+1] < lower):  
                    crossings[j] = 1 
                    break
                #if exit middle back to upper 
                elif (x[j] > lower and x[j+1] > upper):
                    break
                else:
                    j += 1
            i = j+1

        #case 4: lower to middle to upper
        elif (x[i] < lower and x[i+1]> lower):
            j = i+1
            while j < len(x)-1:
                #if middle to upper
                if (x[j] < upper and x[j+1] > upper):
                    crossings[j] = 1
                    break
                #if exit middle back to lower
                elif (x[j] < upper and x[j+1] < lower):
                    break
                else:
                    j += 1
            i = j+1
        else:
            i += 1

    total_crossing = int(sum(crossings))
    return total_crossing, crossings

In [ ]:
#np.random.seed(9)
sigma=10
amplitude=10
T=50

time, traj=SR(amplitude,T,sigma)
total_crossing, crossings = counting(traj)
p=pressure(amplitude,T,time)

plt.plot(time,p/max(p),':',label='pressure')
plt.plot(time,traj,label='position')
plt.xlabel('time (s)')
plt.ylabel('position (µm)')
plt.title("pressure amplitude = {} millibars, sigma = {} V, \n number of hops = {}".format(amplitude,sigma,total_crossing))
plt.legend(loc='upper right')
plt.ylim(-2, 2)
plt.show()

In [ ]:
noise=[0,5,10,15]
fig = plt.figure(figsize = [10,5])
axs = fig.subplots(len(noise),1, sharex=True)
amplitude = 10
T=50

time=np.linspace(0,tot_time, nsteps)
p=pressure(amplitude,T,time)
sync=np.zeros(len(noise))

for ii in range(len(noise)):
    time, traj = SR(amplitude, T, noise[ii],)
    S=(p/max(p)*traj/max(abs(traj)))
    sync[ii]=sum(S)
    axs[ii].plot(time, p/max(p), ':', label = 'pressure (milibars)')
    axs[ii].plot(time, traj, label= 'position (µm)')   
    axs[ii].set_ylabel('position (µm)')
    axs[ii].set_ylim(-2, 2)
    axs[ii].set_title("pressure amplitude = {} millibars, sigma = {} V".format(amplitude, noise[ii]))

axs[-1].set_xlabel('time (s)')
fig.subplots_adjust(top = 0.95, bottom =0.09, left = 0.10, right =0.98, wspace = 0.2, hspace = 0.4)
plt.show()

In [ ]:
noise=range(0,20,2)
amplitude = 10
nsample=30
T=50

time=np.linspace(0,tot_time, nsteps)
p=pressure(amplitude,T,time)
sync=np.zeros(len(noise))

for ii in range(len(noise)):
    S_list=np.zeros(nsample)
    for jj in range(nsample):
        time, traj = SR(amplitude, T, noise[ii],)
        S=(p/max(p)*traj/max(abs(traj)))
        S_list[jj]=sum(S)
    sync[ii]=S_list.mean()

plt.plot(noise, sync)
plt.xlabel('noise level',size=15)
plt.ylabel('Synchronization',size=15)
plt.title("Synchronization rate at different noise levels", size=20)
plt.show()

In [ ]:
plt.scatter(noise, sync/max(sync))
plt.xlabel('noise level',size=15)
plt.ylabel('Synchronization',size=15)
plt.title("Synchronization rate at different noise levels", size=20)
plt.show()